In [62]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import os
import re
import json
from datetime import datetime

In [63]:
# import matplotlib.pylab as pylab
# params = {'legend.fontsize': 18,
#           'figure.figsize': (15, 5),
#          'axes.labelsize': 18,
#          'axes.titlesize':18,
#          'xtick.labelsize':18,
#          'ytick.labelsize':18}
# pylab.rcParams.update(params)

In [64]:
state = 'Rajasthan'
mandi = 'Kota'
exp_no = 52
retrainFreq = 15
actualPricesFilePath = os.path.join(f'../Data/PlottingData/SOYABEAN/ARIMA_Interpolated_Data/', f'{state.upper()}_{mandi.upper()}_Price.csv')
recommendationFilePath = f'./EXP{exp_no}_{state.upper()}_{mandi.upper()}/Recommendations/RetrainFreq{retrainFreq}Day/'
withErrorModelFilePath = os.path.join(recommendationFilePath, 'WithErrorModels')
withoutErrorModelFilePath = os.path.join(recommendationFilePath, 'WithoutErrorModels')
processedFilePath = os.path.join(recommendationFilePath, 'Processed')

### Renaming files

In [65]:
def renameFiles(path, pattern, rpattern):
    comp = re.compile(pattern)
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if os.path.isfile(full_path):
            match = comp.search(f)
            if not match :
                continue

            try:
                new_name = match.expand(rpattern)
                new_name = os.path.join(path, new_name)
            except re.error:
                continue
        
            if os.path.isfile(new_name):
                print('%s -> %s skipped' % (f, new_name))
            else:
                os.rename(full_path, new_name)


In [66]:
pattern, rpattern = r'^recommend_with_errormodel_(\d{0,4}).csv$', r"Day_\1.csv"
renameFiles(withErrorModelFilePath, pattern, rpattern)

In [67]:
pattern, rpattern = r'^recommend_without_errormodel_(\d{0,4}).csv$', r"Day_\1.csv"
renameFiles(withoutErrorModelFilePath, pattern, rpattern)

### Process Recommendations

In [72]:
def processFile(recommendationsPath, actualPath):
    actual_df = pd.read_csv(actualPath, index_col=['DATE'])
    res = pd.DataFrame()
    all_files = os.listdir(recommendationsPath)
    all_files.sort()
    for fileName in all_files:
        print(f"-- Processed {fileName} --")
        path = os.path.join(recommendationsPath, fileName)
        r_df = pd.read_csv(path, index_col=['DATE'])
        a_df = actual_df[actual_df.index.isin(r_df.index)]
        
        curr_day = {}
        curr_day['DATE'] = a_df.index[0]
        curr_day['ACTUAL_PRICE'] = a_df.iloc[0]['PRICE'] # Actual price of day_0
        curr_day['ACTUAL_MAX'] = a_df[['PRICE']].max().iat[0]

        # Without Prospect Theory
        curr_day['MEAN_PRICE'] = r_df.iloc[0]['MEAN_PRICE'] # Mean price of day_0
        curr_day['MAX_MEAN_PRICE'] = r_df[['MEAN_PRICE']].max().iat[0] # Max mean price from day_0..day_29
        curr_day['MAX_MEAN_PRICE_RECOMMEND_DATE'] = r_df[['MEAN_PRICE']].idxmax().iat[0] # Max mean price date from day_0..day_29, recommended date by max value forecast method
        curr_day['MAX_MEAN_RECOMMEND_DAY_PRICE'] = a_df[a_df.index == curr_day['MAX_MEAN_PRICE_RECOMMEND_DATE']]['PRICE'].item() # Actual price on the recommended date by max value forecast method
        # With Prospect Theory
        curr_day['PROSPECT_RECOMMEND_DATE'] = r_df[['PREDICTED']].idxmax().iat[0] # recommended date by prospect theory method
        curr_day['PROSPECT_RECOMMEND_DAY_PRICE'] = a_df.loc[curr_day['PROSPECT_RECOMMEND_DATE']]['PRICE'].item() # Actual price on the recommended date 
        
        curr_df = pd.DataFrame([curr_day])
        res = pd.concat([res, curr_df])
    return res

In [69]:
if not os.path.exists(processedFilePath):
    os.makedirs(processedFilePath)

In [70]:
def seperateForecastMethods(df):
    # Mean value forecast method
    mean_price_df = df[['DATE', 'ACTUAL_PRICE', 'ACTUAL_MAX', 'MAX_MEAN_PRICE_RECOMMEND_DATE', 'MAX_MEAN_RECOMMEND_DAY_PRICE']].copy()
    mean_price_df.rename(columns={'MAX_MEAN_PRICE_RECOMMEND_DATE' : 'RECOMMEND_DATE', 'MAX_MEAN_RECOMMEND_DAY_PRICE' : 'RECOMMEND_DAY_PRICE'} , inplace=True)
    # Prospect theory method
    prospect_df = df[['DATE', 'ACTUAL_PRICE', 'ACTUAL_MAX', 'PROSPECT_RECOMMEND_DATE', 'PROSPECT_RECOMMEND_DAY_PRICE']].copy()
    prospect_df.rename(columns={'PROSPECT_RECOMMEND_DATE' : 'RECOMMEND_DATE', 'PROSPECT_RECOMMEND_DAY_PRICE' : 'RECOMMEND_DAY_PRICE'} , inplace=True)
    return mean_price_df, prospect_df

In [73]:
# Process files with error models
with_error_df = processFile(withErrorModelFilePath, actualPricesFilePath)
with_error_mean_price_df, with_error_prospect_df = seperateForecastMethods(with_error_df)
with_error_mean_price_df.to_csv(os.path.join(processedFilePath, 'with_error_mean_price.csv'), index=False)
with_error_prospect_df.to_csv(os.path.join(processedFilePath, 'with_error_prospect.csv'), index=False)

-- processed Day_2922.csv --


TypeError: reduction operation 'argmax' not allowed for this dtype

In [ ]:
# Process file without error models
without_error_df = processFile(withoutErrorModelFilePath, actualPricesFilePath)
without_error_mean_price_df, without_error_prospect_df = seperateForecastMethods(without_error_df)
without_error_mean_price_df.to_csv(os.path.join(processedFilePath, 'without_error_mean_price.csv'), index=False)
without_error_prospect_df.to_csv(os.path.join(processedFilePath, 'without_error_prospect.csv'), index=False)

### Compute Metrics

In [ ]:
# p_df - ['DATE', 'ACTUAL_PRICE', 'ACTUAL_MAX', 'RECOMMEND_DATE', 'RECOMMEND_DAY_PRICE']
def computeMetrics(p_df):
    ans = {}
    
    n = p_df.shape[0]
    diff = 0
    p_df['DATE'] = pd.to_datetime(p_df['DATE'])
    p_df['RECOMMEND_DATE'] = pd.to_datetime(p_df['RECOMMEND_DATE'])

    for i in range(n-1):
        j = i + 1
        diff += abs((p_df.iloc[i]['RECOMMEND_DATE'] - p_df.iloc[j]['RECOMMEND_DATE']).days)
    
    count = 0
    threshold = 2
    for i in range(n-1):
        j = i + 1
        if abs((p_df.iloc[i]['RECOMMEND_DATE'] - p_df.iloc[j]['RECOMMEND_DATE']).days) <= threshold:
            count += 1
    
    ans['VOR'] = diff / (n-1)
    ans['PCR'] = count*100 / (n-1)
    ans['PAP'] = sum(p_df['ACTUAL_PRICE'] <= p_df['RECOMMEND_DAY_PRICE']) * 100 / n
    ans['NG'] = sum(p_df['RECOMMEND_DAY_PRICE'] - p_df['ACTUAL_PRICE']) / n # Actual price of day 0
    ans['RMSE_ORACLE'] = (sum((p_df['ACTUAL_MAX'] - p_df['RECOMMEND_DAY_PRICE']) ** 2) / n) ** 0.5  
    ans['NG_ORACLE'] = sum(p_df['ACTUAL_MAX'] - p_df['ACTUAL_PRICE']) / n

    return ans

In [ ]:
def generateMetrics(start_date, end_date):
    data = {
        'Model': [],
        'Recommendation Method': [],
        'Metric': [],
        'Value': []
    }

    for fname in os.listdir(processedFilePath):
        if 'netgain' in fname or os.path.isdir(os.path.join(processedFilePath, fname)):
            continue
        p_df = pd.read_csv(os.path.join(processedFilePath, fname))
        p_df = p_df[(p_df['DATE'] >= start_date) & (p_df['DATE'] <= end_date)]
        model = 'ERROR' if 'with_error' in fname else 'WITHOUT_ERROR'
        recommendation_method = 'MAX_MEAN' if 'mean_price' in fname else 'PROSPECT'
        metrics = computeMetrics(p_df)
        data['Model'] += ([model]*len(metrics))
        data['Recommendation Method'] += ([recommendation_method]*len(metrics))
        data['Metric'] += (list(metrics.keys()))
        data['Value'] += (list(metrics.values()))

    metrics_df = pd.DataFrame(data)
    # print(metrics_df['Metric'])
    metrics_df = metrics_df.pivot(index='Metric', columns=['Model', 'Recommendation Method'], values='Value')
    return metrics_df

### Saving computed metrics seasonal and yearly

In [ ]:
for year in range(2014, 2021):
    metricsPath = os.path.join(processedFilePath, 'metrics')
    if not os.path.exists(metricsPath):
        os.makedirs(metricsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    print(f"-- Generating seasonal metrics for period {start_date} to {end_date} --")
    seasonal_df = generateMetrics(start_date, end_date)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    print(f"-- Generating yearly metrics for period {start_date} to {end_date} --")
    yearly_df = generateMetrics(start_date, end_date)
    seasonal_df.to_csv(os.path.join(metricsPath, f'metrics_seasonal_09_{year}_01_{year+1}.csv'))
    yearly_df.to_csv(os.path.join(metricsPath, f'metrics_yearly_{year}.csv'))

-- Generating seasonal metrics for period 2014-09-01 to 2015-01-31 --
-- Generating yearly metrics for period 2014-01-01 to 2014-12-31 --
-- Generating seasonal metrics for period 2015-09-01 to 2016-01-31 --
-- Generating yearly metrics for period 2015-01-01 to 2015-12-31 --
-- Generating seasonal metrics for period 2016-09-01 to 2017-01-31 --
-- Generating yearly metrics for period 2016-01-01 to 2016-12-31 --
-- Generating seasonal metrics for period 2017-09-01 to 2018-01-31 --
-- Generating yearly metrics for period 2017-01-01 to 2017-12-31 --
-- Generating seasonal metrics for period 2018-09-01 to 2019-01-31 --
-- Generating yearly metrics for period 2018-01-01 to 2018-12-31 --
-- Generating seasonal metrics for period 2019-09-01 to 2020-01-31 --
-- Generating yearly metrics for period 2019-01-01 to 2019-12-31 --
-- Generating seasonal metrics for period 2020-09-01 to 2021-01-31 --
-- Generating yearly metrics for period 2020-01-01 to 2020-12-31 --


### Saving aggregated metrics for entire duration

In [ ]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
final_df = generateMetrics(start_date, end_date)
final_df.to_csv(os.path.join(os.path.join(processedFilePath, 'metrics'), f'metrics_2014_2020.csv'))

### Plotting and saving plots seasonal and yearly

In [ ]:
def plotActualForecast(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    final_df = pd.DataFrame()
    for idx in range(start_idx, end_idx, 30):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        final_df = pd.concat([final_df, t_df])
    final_df = final_df[final_df.index <= end_date]
    final_df.index = pd.to_datetime(final_df.index)
    fig, ax = plt.subplots(figsize=(15, 8))
    ax.plot(final_df.index, final_df['PRICE'], label='Actual', color='green')
    ax.plot(final_df.index, final_df['MEAN_PRICE'], label='Forecasted', color='red')
    month_locater = mdates.MonthLocator()
    month_formatter = mdates.DateFormatter("%b")
    ax.xaxis.set_major_locator(month_locater)
    ax.xaxis.set_major_formatter(month_formatter)
    ax.set_xlabel(f'Months')
    ax.set_ylabel('Price(in Rs)')
    ax.set_title(f'Soyabean prices from {start_date} to {end_date}')
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.legend()
    return fig

### Saving yearly and seasonal plots with error models

In [ ]:
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)    
    fig.savefig(os.path.join(plotsPath, f'plot_seasonal_09_{year}_01_{year+1}.png'), facecolor='w')
    plt.close(fig)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
    fig.savefig(os.path.join(plotsPath, f'plot_yearly_{year}.png'), facecolor='w')
    plt.close(fig)

In [ ]:
# plotsPath = os.path.join(processedFilePath, 'plots')
# if not os.path.exists(plotsPath):
#     os.makedirs(plotsPath)
# start_date, end_date = f'2014-01-01', f'2014-12-31'
# fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
# fig.savefig(os.path.join(plotsPath, f'plot_yearly_with_error_2014_2020.png'), facecolor='w')

### Saving yearly and seasonal plots without error models

In [ ]:
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)    
    fig.savefig(os.path.join(plotsPath, f'plot_seasonal_without_error_09_{year}_01_{year+1}.png'), facecolor='w')
    plt.close(fig)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)
    fig.savefig(os.path.join(plotsPath, f'plot_yearly_without_error_{year}.png'), facecolor='w')
    plt.close(fig)

### Generate files with netgain computed on a daily basis

In [ ]:
for fname in os.listdir(processedFilePath):
    if 'with' not in fname:
        continue
    p_df = pd.read_csv(os.path.join(processedFilePath, fname))
    p_df['NET_GAIN'] = p_df['RECOMMEND_DAY_PRICE'] - p_df['ACTUAL_PRICE']
    p_df['NET_GAIN_ORACLE'] = p_df['ACTUAL_MAX'] - p_df['ACTUAL_PRICE']
    p_df.to_csv(os.path.join(processedFilePath, f"netgain_{fname}"), index=False)

### Plotting netgain on a daily basis

In [ ]:
def plotNetGain(start_date, end_date):
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    for fname in os.listdir(processedFilePath):
        if 'netgain' not in fname:
            continue
        p_df = pd.read_csv(os.path.join(processedFilePath, fname))
        p_df = p_df[(p_df['DATE'] >= start_date) & (p_df['DATE'] <= end_date)]
        model = 'with_error' if 'with_error' in fname else 'without_error'
        recommendation_method = 'max_mean' if 'mean_price' in fname else 'prospect'
        fig, ax = plt.subplots(figsize=(15, 8))
        ax.plot(p_df.index, p_df['NET_GAIN'], label='Net Gain', color='green')
        ax.plot(p_df.index, p_df['NET_GAIN_ORACLE'], label='Net Gain Oracle', color='red')
        month_locater = mdates.MonthLocator()
        month_formatter = mdates.DateFormatter("%b")
        ax.xaxis.set_major_locator(month_locater)
        ax.xaxis.set_major_formatter(month_formatter)
        ax.set_xlabel(f'Months')
        ax.set_ylabel('Price(in Rs)')
        ax.set_title(f'Net Gain and Net Gain Oracle from {start_date} to {end_date}')
        for tick in ax.get_xticklabels():
            tick.set_rotation(45)
        ax.legend()
        fig.savefig(os.path.join(plotsPath, f'netgain_{model}_{recommendation_method}_plot_{start_dt.month}_{start_dt.year}_{end_dt.month}_{end_dt.year}.png'), facecolor='w')
        plt.close(fig)

In [ ]:
for year in range(2014, 2021):
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    plotNetGain(start_date, end_date)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    plotNetGain(start_date, end_date)

### Computing RMSE

In [ ]:
# RMSE over interval:
def rmse30DayWindow(df):
    mse = (df["PRICE"] - df["MEAN_PRICE"]) ** 2
    rmse = (mse.mean()) ** .5
    return rmse

def RMSE(df):
    l30, l1, lnormalized = [], [], []
    for i in range(0, len(df), 30):
        x30 = rmse30DayWindow(df[i:i + 30])
        l30.append(x30)
    return np.mean(l30)

In [ ]:
def compute_rmse(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    final_df = pd.DataFrame()
    for idx in range(start_idx, end_idx, 30):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        final_df = pd.concat([final_df, t_df])
    final_df = final_df[final_df.index <= end_date]
    n_df = final_df[['PRICE', 'MEAN_PRICE']].copy()
    rmse = RMSE(n_df)
    daily_rmse = ((n_df['PRICE'] - n_df['MEAN_PRICE']) ** 2).mean() ** .5
    return rmse, daily_rmse

In [ ]:
def compute_daily_rmse(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    all_rmse = []
    for idx in range(start_idx, end_idx):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        t_rmse = ((t_df['PRICE'] - t_df['MEAN_PRICE']) ** 2).mean() ** .5
        all_rmse.append(t_rmse)
    return sum(all_rmse)/len(all_rmse)

In [ ]:
data = {'Type': [], 'Duration': [], 'Year': [], 'Rolling RMSE': [], 'Daily RMSE': []}

In [ ]:
# print("---- With Error Models ----")
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)

    data['Type'].append('Error')
    data['Duration'].append('Seasonal')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)
    # print(f"Seasonal: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")
    
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)    
    # print(f"Yearly: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")
    
    data['Type'].append('Error')
    data['Duration'].append('Yearly')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)

In [ ]:
# print("---- Without Error Models ----")
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    # print(withoutErrorModelFilePath)
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)    
    # print(f"Seasonal: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")

    data['Type'].append('Without Error')
    data['Duration'].append('Seasonal')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)


    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)    
    # print(f"Yearly: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")

    data['Type'].append('Without Error')
    data['Duration'].append('Yearly')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)

In [ ]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
rmse_with, _ = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)  
rmse_without, _ = compute_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)  
print("Rolling RMSE: ")
print(rmse_with, rmse_without)

Rolling RMSE: 
138.23572132962667 149.14806085653055


In [ ]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
rmse_with = compute_daily_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)  
rmse_without = compute_daily_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)  
print("Daily RMSE: ")
print(rmse_with, rmse_without)

Daily RMSE: 
138.22133132902093 146.30596213216836


In [ ]:
rmse_df = pd.DataFrame(data)
rmse_df = pd.pivot(rmse_df, index='Year', columns=['Type', 'Duration'], values=['Rolling RMSE', 'Daily RMSE'])

# print(processedFilePath)
rmse_df.to_csv(os.path.join(os.path.join(processedFilePath, 'metrics'), f'rmse_2014_2020.csv'))
rmse_df.head()

Rolling RMSE                                        Daily RMSE  \
Type            Error             Without Error                   Error   
Duration     Seasonal      Yearly      Seasonal      Yearly    Seasonal   
Year                                                                      
2014       138.349789  163.851176    145.615373  182.784914  168.122926   
2015       146.514513  184.257435    147.845980  176.573554  175.966819   
2016        82.399535  118.797953     87.983493  133.216169   98.927305   
2017       137.246321   73.135883    151.353176   83.353782  110.086259   
2018       157.450510  132.801100    168.220568  142.367504  163.441247   

                                                
Type                 Without Error              
Duration      Yearly      Seasonal      Yearly  
Year                                            
2014      190.987220    164.757964  203.224358  
2015      204.064849    181.982420  195.909729  
2016      130.121424    106.279565  146.457629  
2017       79.861756    124.952693   88.091838  
2018      145.585735    176.800905  156.033963